In [ ]:
import pandas as pd
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from os.path import join
from os import makedirs
from evaluator import Evaluator, CropOffsetFilter
from training_models import Autoencoder, DenoisingAutoencoder, AutoencoderEnsemble
from monitor_models import Autocovariance, AugmentationCrossCovariances, RepresentationCrossCovariances
from metrics import Identity, Variance, CovarianceWeightedAverage

In [ ]:
EXP_NAME = "exp_test"
TRAIN_DATASETS = ["instance", "stead"]
TEST_DATASETS = ["instance", "stead"]
SPLITS = range(0, 5)
EPOCHS = range(0, 20)
METHOD_PARAMS = {}

FILTERS = [CropOffsetFilter(3.0, 3.0, 30.0, 100.0)]

In [ ]:
def calculate_roc_auc(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, epoch, method_params, metric):
    evaluator = Evaluator(EXP_NAME, training_model_class, monitoring_model_class, 
                          train_dataset, test_dataset, FILTERS, epoch, split, method_params, metric)
                    
    tpr, fpr, __ = evaluator.get_roc_vectors()
    roc_auc = auc(fpr, tpr)
    return roc_auc

def model_summary(training_model_class, monitoring_model_class, method_params, metric):
    evaluations = []
    for train_dataset in TRAIN_DATASETS:
        for test_dataset in TEST_DATASETS:
            for split in SPLITS:
                for epoch in EPOCHS:
                    roc_auc = calculate_roc_auc(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, epoch, method_params, metric)
                    evaluations.append({"Training dataset": train_dataset, "Testing dataset": test_dataset, "Split": split, "Epoch": epoch, "ROC-AUC":roc_auc})

    df = pd.DataFrame(evaluations)

    df.loc[:, "Training model"] = training_model_class().name
    df.loc[:, "Monitoring model"] = monitoring_model_class().name

    return df

def grand_summary():
    df_autoencoder_autocovariance = model_summary(Autoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_autocovariance = model_summary(DenoisingAutoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_augmentation_crosscov = model_summary(Autoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_augmentation_crosscov = model_summary(DenoisingAutoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_ensemble_representation_crosscov = model_summary(AutoencoderEnsemble, RepresentationCrossCovariances, 
                                                                    {}, 
                                                                    CovarianceWeightedAverage(input_param="fcov"))

    return pd.concat([df_autoencoder_autocovariance, 
               df_denoisingae_autocovariance, 
               df_autoencoder_augmentation_crosscov,
               df_denoisingae_augmentation_crosscov,
               df_autoencoder_ensemble_representation_crosscov], axis=1)

def method_name(training_model, monitoring_model):
    pass

def plot_rocauc_vs_epochs(df_grand_summary, figsize=(8, 6), ymin = 0.9, ymax=1.0):
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=figsize)
    for train_idx, training_set in enumerate(TRAIN_DATASETS):
        for test_idx, test_set in enumerate(TEST_DATASETS):
            _df = df_grand_summary[(df_grand_summary["Training dataset"] == training_set) & (df_grand_summary["Testing dataset"] == test_set)]

            legend = (train_idx == 1) and (test_idx == 1) 
            sns.lineplot(data=_df, x='Epoch', y='ROC-AUC', hue='Method', ax=axs[train_idx, test_idx], legend=legend)

            axs[train_idx, test_idx].set_ylim(ymin, ymax)
            axs[train_idx, test_idx].set_title(f"Training dataset: {training_set.upper()}, Testing dataset: {test_set.upper()}")

            axs[train_idx, test_idx].get_xaxis().set_visible(train_idx == 1)

            if test_idx == 1:
                axs[train_idx, test_idx].set_ylabel("")

    plt.tight_layout(pad=0)
    plt.savefig('../evals/cross_domain.pdf', dpi=600)